# Exploring First Names and Rankings - 1880-2018


## Overview

The focus of this project is to utilize data from the U.S. Social Security Adminisration to get some hands on practice using Pandas, Numpy and Bokeh. The data set comes coutesy of the SSA and you can read more about it and access it [here](https://catalog.data.gov/dataset/baby-names-from-social-security-card-applications-national-level-data). 

### The Data
The data is split up into one file per year spanning 1880-2018 and includes the relative frequency of first names for U.S. births over the specified time frame. Each data file includes a list of records in the format of "name,sex,number," with the "number" corresponding to the number of occurences of the name & gender combination for the given year. The data is formatted nicely (sorted by sex, then count), so there isn't really much cleaning to do and there shouldn't be any missing values. Here's a quick sample from the first file:

```
Mary,F,7065
Anna,F,2604
Emma,F,2003
...
```
You may be wondering... is this an exhaustive set of names over that time period? In short, the answer is 'No'. For starters, names with fewer than 5 occurrences for a given year are excluded from the list to protect privacy. In addition, only names that are between 2-15 characters are included and sorry Prince... symbols are not allowed. (And yes, I do know Prince's birth name is Prince Rogers Nelson, so cool your jets people.)

If you are interested in more background info, try these pages out:
- Background - https://www.ssa.gov/oact/babynames/background.html
- Main Baby Names Page for SSA - https://www.ssa.gov/oact/babynames/index.html (includes some nice interactive functionality to play with the data (limited to the top 1000 names for each year)


### Analysis Ideas

In this notebook, I'll primarily be using Pandas and Numpy for data manipulation and analysis and Bokeh for visualizations. Starting out, I'm seeking to answer the following questions:

1. What are the most popular names of all time - overall and split by gender? And which names are popular for both males and females (i.e., gender neutral)?
2. What's the most popular male and female name for each decade?
3. Which names consistently rank the highest? And how does this compare to overall popularity?
4. Which letters of the alphabet get the most love when it comes to first initials??

That seems like enough for now... we'll see if any other interesting ideas pop up along the way. To get started, we'll need to read in the data, so let's do that now.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool
# from bokeh.palettes import Spectral4

## Import the Data
To start, let's import a single data file to see what we're working with. Based on the above, we know each data file only has 3 columns - `name`, `gender` and `number`. There are no headers in the data, so we'll have to create those as part of creating the dataframe.

### Create the Initial Dataframe

In [2]:
# Create column headers and read in the first file for 1880
cols = ["name", "gender", "year_count"]
names = pd.read_csv("data/yob1880.txt", names=cols)

# Preview what the data looks like 
print(names.shape)
print(sum(names.year_count))
names.head()

(2000, 3)
201484


,name,gender,year_count
0,Mary,F,7065
1,Anna,F,2604
2,Emma,F,2003
3,Elizabeth,F,1939
4,Minnie,F,1746


So in our initial file, we have 2000 total rows and our data looks as expected. In order to combine the files from all years into a single dataframe, we'll need to add a column for the year so we can keep things straight.

In [3]:
# Add a new column for year
year = 1880
names['year'] = year
names.head()

,name,gender,year_count,year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880


### Read in the Rest of the Data 
Now that we have an initial dataframe ready to go, let's read in the rest of the files and add them to our dataset for analysis. We'll have to account for the new `year` column as we go, which we can get from the file name itself. 

In [4]:
# Concatenate all remaining files into our dataframe

# Set column headers
cols = ["name", "gender", "year_count"]

# Loop through files and append to dataframe (starting with 1881)
for i in range(1881,2019,1):
    filename = str("data/yob" + str(i) + ".txt") 
    temp_df = pd.read_csv(filename, names=cols)
    temp_df["year"] = i
    names = pd.concat([names, temp_df], ignore_index=True) # Reset the index

# Preview the shape of the updated dataframe
print(names.shape)
print(sum(names.year_count))
names.tail()

(1957046, 4)
351653025


,name,gender,year_count,year
1957041,Zylas,M,5,2018
1957042,Zyran,M,5,2018
1957043,Zyrie,M,5,2018
1957044,Zyron,M,5,2018
1957045,Zzyzx,M,5,2018


After combining all the files into one, we can see that we now have nearly **2 million** rows and a total count of over **350 million** which represents the number of unique names accounted for in the data. And that last name looks pretty interesting... ***Zzyzx***. If you want to learn more (I did), you can take a look [here](https://en.wikipedia.org/wiki/Zzyzx,_California), or [here](https://www.babynamewizard.com/baby-name/boy/zzyzx). Moving on...

## Data Analysis
Next, let's do some analysis on our data to answer some of those burning questions we listed out above. Keep in mind, this isn't an exhaustive record containing the names of ***every*** U.S. citizen from 1880-2018. That said, as we saw above, it does account for quite a few people. So hopefully it will be enough to help us spot some interesting trends.

Let's start our analysis with some basics - things like gender split, most popular names (male, female, combined), etc. Before we do that though, let's do a quick check to ensure we don't have any missing values.

### Check for Missing Values

In [5]:
names.isna().sum()

name          0
gender        0
year_count    0
year          0
dtype: int64

OK, looks like we're good to move forward. Next, let's check out the gender split.

### Gender Split - Total
First, let's look at the total gender split to see what that turns up. To do the work, we'll use the pivot_table function to get the view we're after - setting the index to `gender` and the values to `year_count`. 

In [6]:
# Determine the split between Males and Females in the data
names.pivot_table(index="gender",values=["year_count"],aggfunc=sum)

,year_count
gender,
F,174079232
M,177573793


### Gender Split - Over Time
Looks like a pretty even split in the overall numbers. Before we move forward, let's take a look at the split between male and female records for each year to see if there are any interesting trends. Since this data is based on U.S. births where the child has a social security number, we might expect to see a higher number of males in the earlier years with the number of females catching up and surpassing males in the last hundred years. 

First, let's setup Bokeh to output to our notebook so we can view the results inline, then we'll plot it out and see what we get.

In [7]:
# Setup Bokeh to output directly to the notebook
output_notebook(resources=None, verbose=False, hide_banner=True, load_timeout=5000, notebook_type='jupyter')

In [8]:
# Create x and y values from our new temp dataframe
# x values will include the Series of distinct years
x = names['year'].unique()

# y values will be the sum of the count for each year, grouped by gender 
y_male = names[names['gender']=='M'].groupby(['year', 'gender']).year_count.sum() / 1000000
y_female = names[names['gender']=='F'].groupby(['year', 'gender']).year_count.sum() / 1000000

# Create the figure 
p = figure(plot_width=800,plot_height=600, toolbar_location=None, tools="")
p.background_fill_color = "lightslategray"
p.background_fill_alpha = 0.3

# Style the figure
p.title.text="Gender Split Over Time"
p.title.text_color="black"
p.title.text_font="helvetica"
p.title.text_font_style="bold"
p.xaxis.minor_tick_line_color=None
p.yaxis.minor_tick_line_color=None
p.xgrid.grid_line_color = None
p.y_range.start = 0
p.xaxis.axis_label="Year"
p.xaxis.axis_label_text_color="gray"
p.xaxis.axis_label_text_font="helvetica"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label="Count, M"
p.yaxis.axis_label_text_color="gray"
p.yaxis.axis_label_text_font="helvetica"
p.yaxis.axis_label_text_font_style="bold"

# Create a line for each gender
p.line(x, y_male, color="cornflowerblue", line_width=2, alpha=0.8, legend_label=("Male"))
p.line(x, y_female, color="indianred", line_width=2, alpha=0.8, legend_label=("Female"))

p.legend.location = "top_left"

show(p)

Interesting... my hypothesis was pretty much dead wrong. In fact, the data shows almost the complete opposite being true with women outnumbering men for most of the first 60 years and then men overtaking women starting around 1950. It's also interesting that the two lines mirror each other in terms of shape almost perfectly. 

Let's move on to take a look at name popularity.

In [9]:
###TODO###
# Add a chart showing the difference in men and women yoy

### Most Popular Names - Overall
We'll look first at which names are most popular over time for each gender. We can use the `pivot_table()` function again to achieve this result.

In [10]:
# Create a pivot table view for the top 10 names based on total count
names.pivot_table(index=["name", "gender"], values="year_count", aggfunc=sum).sort_values("year_count", ascending=False).head(10)

,,year_count
name,gender,
James,M,5164280
John,M,5124817
Robert,M,4820129
Michael,M,4362731
Mary,F,4125675
William,M,4117369
David,M,3621322
Joseph,M,2613304
Richard,M,2565301


Looks like male names dominate the list of overall popularity. Not sure exactly what that says about male names... maybe they tend to be more traditional, or maybe it's a more frequent occurence that male names are handed down between generations. In any case, we should split this out by gender next so we can see the separate lists. 

### Most Popular Names - By Gender
First, let's create a new dataframe to make this a bit easier. We can use `Pandas.copy()` to make a deep copy of our initial dataframe and then we can collapse the year_count for each name & gender combo.

In [11]:
# Create a copy of the dataframe, drop the year column & rename year_count to total_count
names_copy = names.copy(deep=True)
names_copy = names_copy.drop(columns='year', axis=1)
names_copy = names_copy.rename(columns={'name': 'name', 'gender': 'gender', 'year_count': 'total_count'})

# Collapse the data by name and gender and sum up the year_count column
names_copy = names_copy.groupby(['name', 'gender'], as_index=False)\
            .agg({'total_count': 'sum'}).reindex(columns=names_copy.columns)

#### Top 10 Male Names - All Time

In [12]:
# Filter the new dataframe by gender and sort by year_count
# Top 10 male names based on total count
names_copy[names_copy["gender"]=="M"].sort_values("total_count", ascending=False, ignore_index=True).head(10)

,name,gender,total_count
0,James,M,5164280
1,John,M,5124817
2,Robert,M,4820129
3,Michael,M,4362731
4,William,M,4117369
5,David,M,3621322
6,Joseph,M,2613304
7,Richard,M,2565301
8,Charles,M,2392779
9,Thomas,M,2311849


#### Top 10 Female Names - All Time

In [13]:
# Top 10 female names based on total count
names_copy[names_copy["gender"]=="F"].sort_values("total_count", ascending=False, ignore_index=True).head(10)

,name,gender,total_count
0,Mary,F,4125675
1,Elizabeth,F,1638349
2,Patricia,F,1572016
3,Jennifer,F,1467207
4,Linda,F,1452668
5,Barbara,F,1434397
6,Margaret,F,1248985
7,Susan,F,1121703
8,Dorothy,F,1107635
9,Sarah,F,1077746


### Popular Gender Neutral Names
There's nothing too surprising in the above lists. Let's dig a little deeper and see if we can figure out the most common gender neutral names. In other words, names that are relatively common for both males and females. We can start by looking at the total count for names that appear ***at least*** once for each gender and see what that looks like. I suspect it won't give us exactly what we're after, but let's give it a try. 

To do this analysis, first we'll isolate the list of unique names for each gender into separate Numpy ndarrays and then we'll use `Numpy.intersect1d()` to find the interesection of the two arrays. Once we have the intersection, we can use it to filter our dataframe to the gender neutral names

In [16]:
# Isolate the unique male names & view the total
male_names = names_copy[names_copy['gender'] == 'M'].name.unique()
print("Number of male names: " + str(len(male_names)))

# Isolate the unique female names & view the total
female_names = names_copy[names_copy['gender'] == 'F'].name.unique()                          
print("Number of female names: " + str(len(female_names)))

# Find the intersection of the two arrays & view the total
gn_names = np.intersect1d(male_names, female_names)
print("Number of gender neutral names: " + str(len(gn_names)))

Number of male names: 41475
Number of female names: 67698
Number of gender neutral names: 10773


In [17]:
# Top 10 gender neutral names based on total count
names_copy[names_copy.name.isin(gn_names)]\
    .pivot_table(index="name", values="total_count", aggfunc=sum).sort_values("total_count", ascending=False).head(10)

,total_count
name,
James,5187679
John,5146508
Robert,4840228
Michael,4384463
Mary,4140840
William,4133327
David,3634229
Joseph,2623958
Richard,2574832


Clearly, this result is unexpected as none of the names in the above list is a common ***gender neutral*** name. In fact, if your keeping score at home, you'll notice this list looks suspiciously similar to the list of the most popular names overall. I suspect the result is skewed for names that have a disproportionately high number for one gender and a much smaller number for the other. We can confirm this to be the case by looking at the numbers for one of the names and checking the count for each gender.

In [18]:
# Create a list of the top10 names from the list above
top10_count = names_copy[names_copy.name.isin(gn_names)]\
    .pivot_table(index="name", values="total_count", aggfunc=sum).sort_values("total_count", ascending=False).head(10)
top10_names = top10_count.index
top10_names

Index(['James', 'John', 'Robert', 'Michael', 'Mary', 'William', 'David',
       'Joseph', 'Richard', 'Charles'],
      dtype='object', name='name')

In [19]:
# Use the list to filter out the counts of these names for each gender
names_copy[names_copy.name.isin(top10_names)]\
    .pivot_table(index=["name","gender"], values="total_count").sort_values(["name","gender"]).head(20)

total_count
name    gender             
Charles F             12418
        M           2392779
David   F             12907
        M           3621322
James   F             23399
        M           5164280
John    F             21691
        M           5124817
Joseph  F             10654
        M           2613304
Mary    F           4125675
        M             15165
Michael F             21732
        M           4362731
Richard F              9531
        M           2565301
Robert  F             20099
        M           4820129
William F             15958
        M           4117369

Just as I suspected, each name in the list has a disproportionately high count for one gender and a fairly small count for the other. In order to find a more representative list, we'll have to do some more clean up. Let's start by creating a new df with the gender neutral names and then filtering it to a threshold count for each gender. Then, we can revisit the top 10 list to see if we get a different result.

In [20]:
###TODO###
# figure out if there is a way to refactor this to avoid creating a new df
# create a function to adjust the threshold by passing in a number and returning the list

In [23]:
# Compile a list of names that are not likely to be common 
# gender neutral names by generating a list of names where count is < 30k
non_gn_names = names_copy.name[names_copy['total_count'] < 30000]

# Create a copy of our name_counts dataframe 
gn_name_counts = names_copy.copy(deep=True)
print("Rows before: " + str(gn_name_counts.shape[0]))

# Drop rows that don't meet the threshold
gn_name_counts.drop(gn_name_counts[gn_name_counts['name'].isin(non_gn_names)].index, inplace = True) 

print("Rows after: " + str(gn_name_counts.shape[0]))

Rows before: 109173
Rows after: 79


In [24]:
# And now, pivot and display the top 10 based on total count
gn_name_counts.pivot_table(index="name", values="total_count", aggfunc=sum).sort_values("total_count", ascending=False).head(10)

,total_count
name,
Willie,595102
Kelly,553154
Terry,519811
Jordan,505517
Taylor,430836
Alexis,401937
Leslie,379807
Jamie,353733
Shannon,347023


### TODO - Most Popular Names by Decade 

### TODO - Most Consistent Highly Ranked Names (+ compared to overall top 10)

### Most Popular First Initial
Now let's go back to our original dataframe and see if we can figure out the most popular first initial by gender and also combined.

In [25]:
# Create a simple function to get the first letter of each name
def getFirstLetter(string):
    """Function to return first letter of a string
    
        Args:
            string
        
        Returns: 
            first letter
            
    """
    return string[0]

# Apply function to dataframe to get first letter for each name
names["letter"] = names.name.apply(getFirstLetter)
names.head()

,name,gender,year_count,year,letter
0,Mary,F,7065,1880,M
1,Anna,F,2604,1880,A
2,Emma,F,2003,1880,E
3,Elizabeth,F,1939,1880,E
4,Minnie,F,1746,1880,M


In [26]:
# The x values are the series of letters 
letters = names['letter'].unique().tolist()
letters.sort()

# The y values are the sum of the count for each letter 
counts = round((names.groupby(['letter']).year_count.sum() / 1000000), 2)
counts = counts.tolist()

# Create the figure 
p = figure(x_range=letters, plot_height=400, title="Letter Counts",
           toolbar_location=None, tools="")
p.background_fill_color = "lightslategray"
p.background_fill_alpha = 0.3

# Style the figure
p.title.text="First Initial Popularity"
p.title.text_color="black"
p.title.text_font="helvetica"
p.title.text_font_style="bold"
p.xaxis.minor_tick_line_color=None
p.yaxis.minor_tick_line_color=None
p.xgrid.grid_line_color = None
p.xaxis.axis_label="Letter"
p.xaxis.axis_label_text_color="gray"
p.xaxis.axis_label_text_font="helvetica"
p.xaxis.axis_label_text_font_style="bold"
p.yaxis.axis_label="Count, M"
p.yaxis.axis_label_text_color="gray"
p.yaxis.axis_label_text_font="helvetica"
p.yaxis.axis_label_text_font_style="bold"

p.vbar(x=letters, top=counts, width=0.6, color="cornflowerblue", alpha=0.9,)
p.y_range.start = 0

show(p)

### Write to csv
Let's write this data to a single csv to make it easier to import in the future.

In [ ]:
# Write new dataframe to single csv file
# names.to_csv('names.csv') 

## TODO
- Create function to plot the popularity of a single name over time
    - Call with my name
    - Check for popularity bump of unique names (e.g. Barrack, Kanye, etc.)
- Add function to generate the following stats for a specified name and yob:
    - Rank in yob
    - Highest ranking year / rank
    - Lowest ranking year / rank


**Some Ideas for an Interactive UI**
1. Enter a name and year of birth to get statistics:
    1. How popular was the name during the specified yob?
    2. What were the most popular names for yob (M/F)?
    3. Which years did the name rank highest and lowest? 
2. Baby name generator ideas:
    1. Throwback name ideas (i.e. popular names from a specified period - n years in the past)
    2. Popular gender neutral names - all time, last 10 years, etc.
    3. Specify a first initial and gender to get a list of popular or random names
    4. "I'm feeling lucky" - Generates a random first & middle name combo (could include some controls for specifying the acceptable popularity levels)